In [7]:
import pandas as pd
import dvc.api

In [2]:
from copy import deepcopy
import hashlib
from collections import OrderedDict
import json
def sortOD(od):
    res = OrderedDict()
    for k, v in sorted(od.items()):
        if isinstance(v, dict):
            res[k] = sortOD(v)
        else:
            res[k] = deepcopy(v)
    return res

In [51]:
df_metadata = pd.read_csv('../attribute_extraction_metadata_template/Initial Attribute Definition for First Release - UPDATED SHEET .csv')

In [53]:
df_metadata = df_metadata[['category', 'category_id', 'attribute_field', 'category_attributevalue', 'description', 'max_multi_select', 'usage', 'entry mode']]

In [54]:
metadata_category = set(df_metadata.category)
metadata_category_ids = set(df_metadata.category_id)

In [55]:
category2config = {}
for i in metadata_category:
    df_tmp = df_metadata[df_metadata.category == i]
    category2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])
categoryid2config = {}
for i in metadata_category_ids:
    df_tmp = df_metadata[df_metadata.category_id == i]
    categoryid2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])

In [15]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [19]:
tax2id = {}
for i in df_tax.to_dict('records'):
    if len(i['category_path']) > 0:
        tax2id[i['category_path']] = int(i['id'])

# sample file

In [56]:
df_samples = pd.read_csv('sdt887_product_attribution_data_one_listing_per_leaf_node__20230111.csv')

In [57]:
df_samples.head(2)

,product_id,category_id,category_name,category_path,L1,L2,num_extra_photos,title,product_description,main_image_url
0,60f1d9657dd37ab3cbb26868,5504,Axes,Tools > Hand Tools > Axes,Tools,Tools > Hand Tools,9,"Axe holder, axe stand, Leviathan axe, God of W...","Axe holder, axe stand, Leviathan axe, God of W...",https://canary.contestimg.wish.com/api/webimag...
1,6184dc13dfb278fe92818499,2939,Pastry Brushes,"Home & Garden > Kitchen, Dining & Bar > Bakewa...",Home & Garden,"Home & Garden > Kitchen, Dining & Bar",8,New silicone baking pastry tool oil brush high...,"Welcome to my shop, I wish you a happy shoppin...",https://canary.contestimg.wish.com/api/webimag...


In [7]:
df_samples = df_samples[df_samples.category_path.apply(lambda x: x in metadata_category)]
df_samples = df_samples.sample(10, random_state=42)

In [10]:
res = []
for i in df_samples[['product_id', 'category_id', 'category_path', 'title', 'product_description', 'main_image_url']].to_dict('records'):
    i['text'] = \
f"""Title: {i['title']}
Description: {i['product_description']}"""
    i['attribute_config'] = category2config[i['category_path']]
    task_json = json.dumps(sortOD(i)).encode('utf-8')
    task_id = hashlib.md5(task_json).hexdigest()
    i['task_id'] = f"product_attribution_md_{task_id}"
    i['preloading_result'] = None
    res.append(i)

In [11]:
pd.DataFrame(res).to_json('example_offshore_input_listing_attr_extract_v2.json', lines=True, orient='records')

# demo file

In [58]:
df_samples_orig_text = pd.read_json(dvc.api.get_url(
    'datasets/data/wish_attr_extract_label/appen/input_batch/appen_product_attribution_batch1_text.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)
df_samples_orig_textimg = pd.read_json(dvc.api.get_url(
    'datasets/data/wish_attr_extract_label/appen/input_batch/appen_product_attribution_batch2_text_image.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [59]:
pid2cid = {}
for i in df_samples_orig_text[['product_id', 'category_id']].to_dict('records'):
    pid2cid[i['product_id']] = int(i['category_id'])
for i in df_samples_orig_textimg[['product_id', 'category_id']].to_dict('records'):
    pid2cid[i['product_id']] = int(i['category_id'])

In [60]:
df_samples = pd.read_csv(dvc.api.get_url(
    'datasets/data/wish_attr_extract_label/appen/input_batch_processed/appen_product_attribution_batch1.csv',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
))

In [61]:
df_samples = df_samples[df_samples.product_id.apply(lambda x: x in pid2cid)]

In [62]:
df_samples['category_id'] = df_samples['product_id'].apply(lambda x: pid2cid[x])

In [64]:
df_samples = df_samples[df_samples.category_id.apply(lambda x: x in categoryid2config)]
df_samples = df_samples.sample(10, random_state=42)

In [66]:
res = []
for i in df_samples[['product_id', 'category_id', 'category_path', 'title', 'product_description', 'main_image_url']].to_dict('records'):
    i['text'] = \
f"""Title: {i['title']}
Description: {i['product_description']}"""
    i['attribute_config'] = categoryid2config[i['category_id']]
    task_json = json.dumps(sortOD(i)).encode('utf-8')
    task_id = hashlib.md5(task_json).hexdigest()
    i['task_id'] = f"product_attribution_md_{task_id}"
    i['preloading_result'] = None
    res.append(i)

In [68]:
pd.DataFrame(res).to_json('offshore_md/demo_product_attr_extract_md_02272023.json', lines=True, orient='records')